In [15]:
from selenium import webdriver
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
url = "https://one.bees.com/order-management/active-orders"
email = input('Email:')
senha = input('Senha:')
filial = input('Filial')

driver = webdriver.Chrome()
driver.maximize_window()
wait = WebDriverWait(driver,20)

driver.get(url)
campo_email = wait.until(EC.element_to_be_clickable(('id','signInName')))
campo_email.send_keys(email)
botao_continue = wait.until(EC.element_to_be_clickable(('id','next')))
botao_continue.click()
campo_senha = wait.until(EC.element_to_be_clickable(('id','password')))
campo_senha.send_keys(senha)
botao_continue = wait.until(EC.element_to_be_clickable(('id','next')))
botao_continue.click()
botao_entendi = wait.until(EC.element_to_be_clickable(('xpath','//button[text()="Entendi"]')))
botao_entendi.click()
time.sleep(5)


In [ ]:
import pandas as pd
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time

# Função para coletar dados dos pedidos
def coletar_dados_pedidos(driver):
    order_data = []
    # Espera até que a tabela de pedidos esteja visível
    tbody = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]'))
    )
    # Percorre todas as linhas da tabela de pedidos
    for tr in tbody.find_elements(By.XPATH, './/tr'):
        order = tr.find_elements(By.XPATH, './/td[1]')[0].text.strip()
        data_pedido = tr.find_elements(By.XPATH, './/td[2]')[0].text.strip()
        data_entrega = tr.find_elements(By.XPATH, './/td[3]')[0].text.strip()
        responsavel = tr.find_elements(By.XPATH, './/td[4]')[0].text.strip()

        # Adiciona os dados extraídos à lista
        order_data.append({
            'Numero Pedido': order,
            'Data Pedido': data_pedido,
            'Data Entrega': data_entrega,
            'Responsavel': responsavel
        })
    # Cria o DataFrame a partir dos dados extraídos
    return pd.DataFrame(order_data)

# Função para navegar pelas páginas de pedidos
def navegar_paginas(driver):
    """Navega por todas as páginas de pedidos e coleta os dados de cada uma."""
    print("Navegando pelas páginas de pedidos...")
    page_number = 1
    df_total = pd.DataFrame()  # DataFrame para armazenar todos os pedidos

    while True:
        try:
            print(f"Coletando dados da página {page_number}...")

            # Coleta dados da página atual
            df = coletar_dados_pedidos(driver)
            df_total = pd.concat([df_total, df], ignore_index=True)

            # Tenta localizar e clicar no botão "Próxima Página"
            page_xpath = f'//li[@title="page {page_number}"]'  # Ajuste o XPath conforme necessário
            next_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, page_xpath))
            )
            next_button.click()

            # Espera até que a tabela da próxima página seja carregada
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]'))
            )

            # Aumenta o número da página
            page_number += 1
            time.sleep(2)  # Atraso de 2 segundos para dar tempo para a próxima página carregar

        except Exception as e:
            print(f"Não há mais páginas.")
            break

    return df_total

# Função para coletar os detalhes de cada pedido
def coletar_detalhes(driver, df):
    """Coleta os detalhes dos produtos para cada pedido, evitando duplicação de coleta."""
    detalhes = []
    pedidos_processados = set()  # Conjunto para armazenar pedidos já processados

    for order in df['Numero Pedido']:
        if order in pedidos_processados:
            continue  # Ignora o pedido se já foi processado

        try:
            print(f"Coletando detalhes para o pedido {order}...")

            # Acessa a página de detalhes do pedido
            driver.get(f'https://one.bees.com/order-management/active-orders/{order}')
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//tbody[@role="rowgroup"]'))
            )
            time.sleep(3)
            # Tenta clicar no botão 'Ver mais' se presente
            try:
                vermais_button = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/a')
                if vermais_button:
                    vermais_button[0].click()
            except Exception as e:
                continue

            # Coleta dados adicionais do pedido
            data_entrega = df[df['Numero Pedido'] == order]['Data Entrega'].values[0]
            responsavel = df[df['Numero Pedido'] == order]['Responsavel'].values[0] 

            # Coleta os dados dos produtos (caso existam)
            WebDriverWait(driver, 10).until(
                EC.presence_of_element_located((By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[9]/div[3]/div/div/table/tbody'))
            )
            tbody = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[9]/div[3]/div/div/table/tbody')
            produtos = tbody.find_elements(By.XPATH, './/tr[@role="row"]')

            for produto in produtos:
                nome_sku = produto.find_element(By.XPATH, './/td[1]//div/div[2]/p[1]').text.strip()
                sku = produto.find_element(By.XPATH, './/td[1]//div/div[2]/p[2]').text.strip()
                preco = produto.find_element(By.XPATH, './/td[1]//div/div[2]/div/p').text.strip()
                quantidade_pedida = produto.find_element(By.XPATH, './/td[2]').text.strip()
                quantidade_preparar = produto.find_element(By.XPATH, './/td[3]').text.strip()

                # Coletando dados do pedido
                _numero_pedido = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[2]/div[1]/h2').text.strip().replace('Pedido #', '')
                cd = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[2]/p[2]').text.strip()
                documento = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[2]/p[2]').text.strip()

                # Coletando dados de contato e outras informações
                try:
                    ie_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[2]/p[3]')
                    ie = ie_element[0].text.strip() if ie_element else None
                except Exception as e:
                    ie_element = "ISENTO"
                    if ie_element == None:"ISENTO"
                nome_comercial = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[3]/p[2]').text.strip()
                cidade_uf = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[3]').text.strip()
                endereco = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[2]').text.strip()
                cep = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[5]/p[4]').text.strip()
                _data_pedido = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[2]/div[1]/p[2]').text.strip()
                forma_pgto = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[2]/div[2]/p[2]').text.strip()
                status = driver.find_element(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[4]/div[1]/p[2]').text.strip()

                try:
                    contato1_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[2]')
                    contato1 = contato1_element[0].text.strip() if contato1_element else "teste@teste.com"
                except Exception as e:
                    contato1 = "teste@teste.com"
                    print(f"Erro ao coletar contato2 para o pedido {contato1}: {e}")
                try:

                    contato2_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[3]')
                    contato2 = contato2_element[0].text.strip() if contato2_element else None
                except Exception as e:
                    contato2 = None
                    print(f"Erro ao coletar contato2 para o pedido {contato2}: {e}")
                try:

                    contato3_element = driver.find_elements(By.XPATH, '//*[@id="single-spa-application:@supplier-portal/order-management-beta-mfe"]/div/div[8]/div[3]/div[1]/div[4]/p[4]')
                    contato3 = contato3_element[0].text.strip() if contato3_element else None
                except Exception as e:
                    contato3 = None
                    print(f"Erro ao coletar contato2 para o pedido {contato3}: {e}")

                detalhes.append({
                    "Documento": documento,
                    "IE": ie,
                    "Forma de Pagamento": forma_pgto,
                    "SKU": sku,
                    "Preço": preco,
                    "Quantidade Pedida": quantidade_pedida,
                    "Responsavel": responsavel,
                    "Nome Comercial": nome_comercial,
                    "Contato 1": contato1,
                    "Contato 2": contato2,
                    "Contato 3": contato3,
                    "Numero Pedido": _numero_pedido,
                    "Centro de Distribuição": cd,
                    "Status": status,
                    "Nome do Produto": nome_sku,
                    "Quantidade Preparar": quantidade_preparar,
                    "CEP": cep,
                    "Endereço de Entrega": endereco,
                    "Cidade/UF": cidade_uf,
                    "Data Pedido": _data_pedido,
                    "Data Entrega": data_entrega
                })

                pedidos_processados.add(order)  # Marca o pedido como processado

        except Exception as e:
            print(f"Erro ao coletar detalhes para o pedido {order}")
            continue
    
    return pd.DataFrame(detalhes)

# Função principal para rodar o processo
def main(driver, filial):
    df_pedidos = navegar_paginas(driver)  # Coleta os dados dos pedidos
    df_detalhes = coletar_detalhes(driver, df_pedidos)  # Coleta os detalhes de cada pedido
    caminho = r'G:\Drives compartilhados\Cadastro BEES'
    df_detalhes.to_csv(f'{caminho}/Pedidos_A_Preparar_{filial}.csv', index=False, encoding='utf-8-sig')    
    return df_detalhes
    
df_final = main(driver, filial)


Navegando pelas páginas de pedidos...
Coletando dados da página 1...
Coletando dados da página 2...
Não há mais páginas.
Coletando detalhes para o pedido 9197135876...
Coletando detalhes para o pedido 9197133359...
Coletando detalhes para o pedido 9197132881...


In [14]:
df_final['Contato 1'] = df_final['Contato 1'].str.replace(r"[()\s-]", "", regex=True)
df_final['Contato 2'] = df_final['Contato 2'].str.replace(r"[()\s-]", "", regex=True)

caminho = r'G:\Drives compartilhados\Cadastro BEES'
pedidos = df_final.to_csv(f'{caminho}/Pedidos_A_Preparar_{filial}.csv', index=False, encoding='utf-8-sig')
df_final

,Documento,IE,Forma de Pagamento,SKU,Preço,Quantidade Pedida,Responsavel,Nome Comercial,Contato 1,Contato 2,...,Numero Pedido,Centro de Distribuição,Status,Nome do Produto,Quantidade Preparar,CEP,Endereço de Entrega,Cidade/UF,Data Pedido,Data Entrega
0,CPF: 09932087700,INSCRICAO_ESTADUAL: ISENTO,Pix na entrega,178,$55.8,2,SINVAL JOSE DA SILVA,BAR DO JHOOL,zeniltonpinto050@gmail.com,21985252558,...,9197135876,RIGARRRJCAPITAL,A ser preparado,Rum Montilla Carta Ouro Garrafa Vidro 1L,2,22710025,"R Vila Aurora, 1000, Casa 68, CURICICA","RIO DE JANEIRO, RJ","14 de jan. de 2025, 18:38",18/01/2025
1,CPF: 09932087700,INSCRICAO_ESTADUAL: ISENTO,Pix na entrega,69,$55.8,2,SINVAL JOSE DA SILVA,BAR DO JHOOL,zeniltonpinto050@gmail.com,21985252558,...,9197135876,RIGARRRJCAPITAL,A ser preparado,Rum Montilla Carta Branca Garrafa Vidro 1L,2,22710025,"R Vila Aurora, 1000, Casa 68, CURICICA","RIO DE JANEIRO, RJ","14 de jan. de 2025, 18:38",18/01/2025
2,CPF: 09932087700,INSCRICAO_ESTADUAL: ISENTO,Pix na entrega,2089,$119.4,6,SINVAL JOSE DA SILVA,BAR DO JHOOL,zeniltonpinto050@gmail.com,21985252558,...,9197135876,RIGARRRJCAPITAL,A ser preparado,Coquetel Bananinha Reggiani Garrafa Vidro 920ml,6,22710025,"R Vila Aurora, 1000, Casa 68, CURICICA","RIO DE JANEIRO, RJ","14 de jan. de 2025, 18:38",18/01/2025
3,CPF: 09932087700,INSCRICAO_ESTADUAL: ISENTO,Pix na entrega,290,$41.8,2,SINVAL JOSE DA SILVA,BAR DO JHOOL,zeniltonpinto050@gmail.com,21985252558,...,9197135876,RIGARRRJCAPITAL,A ser preparado,Conhaque Dreher Garrafa Vidro 900ml,2,22710025,"R Vila Aurora, 1000, Casa 68, CURICICA","RIO DE JANEIRO, RJ","14 de jan. de 2025, 18:38",18/01/2025
4,CPF: 09932087700,INSCRICAO_ESTADUAL: ISENTO,Pix na entrega,71,$83.4,6,SINVAL JOSE DA SILVA,BAR DO JHOOL,zeniltonpinto050@gmail.com,21985252558,...,9197135876,RIGARRRJCAPITAL,A ser preparado,Aguardente Old Cesar 88 Garrafa Vidro 965ml,6,22710025,"R Vila Aurora, 1000, Casa 68, CURICICA","RIO DE JANEIRO, RJ","14 de jan. de 2025, 18:38",18/01/2025
5,CPF: 09932087700,INSCRICAO_ESTADUAL: ISENTO,Pix na entrega,1518,$59.4,6,SINVAL JOSE DA SILVA,BAR DO JHOOL,zeniltonpinto050@gmail.com,21985252558,...,9197135876,RIGARRRJCAPITAL,A ser preparado,"Coquetel Cantina da Serra Garrafa Pet 1,5L",6,22710025,"R Vila Aurora, 1000, Casa 68, CURICICA","RIO DE JANEIRO, RJ","14 de jan. de 2025, 18:38",18/01/2025
6,CPF: 13456662769,INSCRICAO_ESTADUAL: ISENTO,Pix na entrega,2851,$1690.8,12,GRAZIELLA SIQUEIRA,DOUTOR BEBIDAS,arquitiurysantos@gmail.com,21987395822,...,9197133359,RIGARRRJCAPITAL,A ser preparado,Whisky Jack Daniels Apple Garrafa Vidro 1L,12,24800001,"ANTONIO ALVES VIANA, 02, LT 3 LJ 2 PX AO HABIB...","ITABORAI, RJ","14 de jan. de 2025, 18:18",18/01/2025
7,CPF: 14092685785,INSCRICAO_ESTADUAL: ISENTO,Dinheiro,2851,$3381.6,24,WELLINGTON COELHO BAPTISTA DA SILVA,WELLINGTON COELHO BA,testes@testes.com.br,21970178357,...,9197132881,RIGARRRJCAPITAL,A ser preparado,Whisky Jack Daniels Apple Garrafa Vidro 1L,24,25268250,"R A, 23, LJ A, NOVA CAMPINAS","DUQUE DE CAXIAS, RJ","14 de jan. de 2025, 18:13",18/01/2025
8,CNPJ: 52198004000120,INSCRICAO_ESTADUAL: 125682295117,Cartão de Crédito na Entrega (Somente em 1x),3470,$175.6,4,NOVA EMPORIO CONQUISTA PADARIA E MERCADO,NOVA EMPORIO CONQUIS,11965678021,None,...,9197127156,RIGARRSPCAPITAL,A ser preparado,Cachaça Seleta Ouro Garrafa Vidro 1L,4,03213000,"ANTENAS, 229, VILA CALIFORNIA","SAO PAULO, SP","14 de jan. de 2025, 17:51",18/01/2025
9,CPF: 11840965703,INSCRICAO_ESTADUAL: ISENTO,Dinheiro,2028,$43.9,1,MARIA JOSE DE OLIVEIRA SILVA,MARIA BAR,21986068621,21036598633,...,9197125554,RIGARRRJCAPITAL,A ser preparado,Whisky White Horse Garrafa Vidro 500ml,1,26321140,"DOUTOR HOMERO, 20, qd 06 casa 02, JARDIM QUEIM...","QUEIMADOS, RJ","14 de jan. de 2025, 17:41",18/01/2025


In [12]:
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
wait = WebDriverWait(driver, 30)

cidades_interior = ['ITATIAIA', 'RESENDE', 'PORTO REAL', 'QUATIS', 'BARRA MANSA', 'VOLTA REDONDA', 'ARROZAL', 'PINHEIRAL', 'PIRAI', 'BARRA DO PIRAI', 'MENDES', 'VASSOURA', 'RIO CLARO']
cidades_interior2 = ['MANGARATIBA', 'ANGRA DOS REIS', 'PARATY']
driver.get("https://web.whatsapp.com/")
while len(driver.find_elements(By.ID, "side")) < 1:
    time.sleep(1)
time.sleep(5)
driver.find_element('xpath','//span[@data-icon="settings-outline"]').click()
time.sleep(3)
driver.find_element('xpath','//div[text()="Conversas"]').click()
time.sleep(3)
driver.find_element('xpath','//*[@id="app"]/div/div[3]/div/div[2]/div[1]/span/div/span/div/div/div/div/div/div[2]/div[6]/div/div').click()
time.sleep(3)
driver.find_element('xpath','//*[@id="app"]/div/div[3]/div/header/div/div/div/div/span/div/div[1]/div[1]/button/div/span').click()
time.sleep(3)

def enviar_mensagem_pedido(driver, grupo, pedido):

    estabelecimento = grupo['Nome Comercial'].iloc[0]
    contato1 = grupo['Contato 1'].iloc[0]
    contato2 = grupo['Contato 2'].iloc[0]
    contato3 = grupo['Contato 3'].iloc[0]
    forma_pgto = grupo['Forma de Pagamento'].iloc[0]
    endereco = grupo['Endereço de Entrega'].iloc[0]
    cidade_uf = grupo['Cidade/UF'].iloc[0]
    cep = grupo['CEP'].iloc[0]
    documento = grupo['Documento'].iloc[0]
    ie = grupo['IE'].iloc[0]
    data_entrega = grupo['Data Entrega'].iloc[0]
    responsavel = grupo['Responsavel'].iloc[0]


    mensagem = f"Novo Pedido *{pedido}*\n*Responsavel*: {responsavel}\n*Estabelecimento*: {estabelecimento}\n*Contato 1*: {contato1}\n*Contato 2*: {contato2}\n*Contato 3*: {contato3}\n*Forma de Pagamento*: {forma_pgto}\n*Endereço de Entrega*: {endereco}\n*Cidade/UF*: {cidade_uf}\n*CEP*: {cep}\n{documento}\n{ie}\n*Data Entrega*: {data_entrega}\n"
    
    for _, row in grupo.iterrows():
        mensagem += f"*Produto*: {row['Nome do Produto']} | Quantidade: {row['Quantidade Pedida']} | Preço: R{row['Preço']}\n"

    return mensagem 

def verificar_condicoes_e_enviar_mensagem(driver, df_final):
    for pedido, grupo in df_final.groupby('Numero Pedido'):
        cidade_uf = grupo['Cidade/UF'].iloc[0]
        documento = grupo['Documento'].iloc[0]

  
        mensagem = enviar_mensagem_pedido(driver, grupo, pedido)

        if not any(cidade in cidade_uf for cidade in cidades_interior) and "CPF" in documento and "RJ" in cidade_uf:
          
            field_find = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[1]')))
            field_find.send_keys("NF PEDIDOS VENDAS bebida in box Bees")
            field_find.send_keys(Keys.ENTER)
            field_message = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[2]')))
            field_message.send_keys(mensagem)
            botao_enviar = driver.find_element(By.XPATH, "//button[@aria-label='Enviar']")
            botao_enviar.click()

        # #if any(cidade in cidade_uf for cidade in cidades_interior) and "CPF" in documento and "RJ" in cidade_uf:
          
        #     field_find = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[1]')))
        #     field_find.send_keys("Pedidos BEES - BIB Angra")
        #     field_find.send_keys(Keys.ENTER)
        #     field_message = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[2]')))
        #     field_message.send_keys(mensagem)
        #     botao_enviar = driver.find_element(By.XPATH, "//button[@aria-label='Enviar']")
        #     botao_enviar.click()
        # if any(cidade in cidade_uf for cidade in cidades_interior2) and "CNPJ" in documento and "RJ" in cidade_uf:
          
        #     field_find = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[1]')))
        #     field_find.send_keys("Pedidos BEES - BIB Angra")
        #     field_find.send_keys(Keys.ENTER)
        #     field_message = wait.until(EC.presence_of_element_located(('xpath','(//div[@role="textbox"])[2]')))
        #     field_message.send_keys(mensagem)
        #     botao_enviar = driver.find_element(By.XPATH, "//button[@aria-label='Enviar']")
            botao_enviar.click()

verificar_condicoes_e_enviar_mensagem(driver, df_final)
time.sleep(5)

StaleElementReferenceException: Message: stale element reference: stale element not found in the current frame
  (Session info: chrome=131.0.6778.265); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#stale-element-reference-exception
Stacktrace:
	GetHandleVerifier [0x00007FF608A680D5+2992373]
	(No symbol) [0x00007FF6086FBFD0]
	(No symbol) [0x00007FF60859590A]
	(No symbol) [0x00007FF60859C46F]
	(No symbol) [0x00007FF60859F11C]
	(No symbol) [0x00007FF60859F1EF]
	(No symbol) [0x00007FF6085EB3FB]
	(No symbol) [0x00007FF6085DCC36]
	(No symbol) [0x00007FF60860F36A]
	(No symbol) [0x00007FF6085DC596]
	(No symbol) [0x00007FF60860F580]
	(No symbol) [0x00007FF60862F584]
	(No symbol) [0x00007FF60860F113]
	(No symbol) [0x00007FF6085DA918]
	(No symbol) [0x00007FF6085DBA81]
	GetHandleVerifier [0x00007FF608AC6A2D+3379789]
	GetHandleVerifier [0x00007FF608ADC32D+3468109]
	GetHandleVerifier [0x00007FF608AD0043+3418211]
	GetHandleVerif

In [10]:
import pandas as pd
import re
import urllib.parse
import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

# Função para limpar e formatar números de telefone
def clean_phone_numbers(row):
    """Extrai, limpa e formata números de telefone das colunas Contato 1, 2 e 3."""
    cleaned_numbers = set()  # Usar um set para evitar números duplicados
    for column in ['Contato 1', 'Contato 2', 'Contato 3']:
        if pd.isna(row[column]):  # Verificar se a célula está vazia
            continue
        phone_numbers = str(row[column]).split(',')  # Divide por vírgula caso haja múltiplos números
        for number in phone_numbers:
            number = re.sub(r'[^\d]', '', number)  # Remove caracteres especiais, como espaços e hífens
            if number:  # Adiciona o número se não for vazio
                cleaned_numbers.add(str(number))  # Garante que o número seja tratado como string
    return list(cleaned_numbers)  # Retorna a lista de números de telefone limpos

# Função para aguardar o carregamento completo da conversa
def aguardar_carregamento(driver):
    try:
        # Espera até que o campo de mensagem esteja visível e interativo
        WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.XPATH, '//div[@contenteditable="true"]'))
        )
        print("Conversa carregada com sucesso.")
    except Exception as e:
        driver.refresh()



print("Aguardando conexão com o WhatsApp Web...")

while len(driver.find_elements(By.ID, "side")) < 1:
    time.sleep(1)

print("Conectado ao WhatsApp Web.")

# Carregar os dados da planilha de pedidos com os detalhes dos pedidos
tabela_pedidos = pd.read_csv(f'G:/Drives compartilhados/Cadastro BEES/Pedidos_A_Preparar_{filial}.csv')

# Conjunto de telefones processados para evitar envios duplicados na execução atual
telefones_processados = set()

# Enviar mensagens para pedidos pendentes
for index, row in tabela_pedidos.iterrows():
    if row['Status'] == 'Enviado':  # Verifica se o pedido já foi enviado
        continue

    numero_pedido = row['Numero Pedido']

    # Obter os números de telefone das três colunas
    telefones = clean_phone_numbers(row)

    # Verificar se algum telefone já foi processado nesta execução
    if any(phone in telefones_processados for phone in telefones):
        print(f"Algum número do pedido {numero_pedido} já foi processado. Ignorando envio.")
        continue

    # Buscar os dados do pedido na tabela
    pedido_data = tabela_pedidos[tabela_pedidos['Numero Pedido'] == numero_pedido]
    endereco = pedido_data.iloc[0]['Endereço de Entrega']
    cep = pedido_data.iloc[0]['CEP']
    cidade_uf = pedido_data.iloc[0]['Cidade/UF']
    forma_pagamento = pedido_data.iloc[0]['Forma de Pagamento']

    # Criar lista de produtos do pedido
    produtos = ""
    for _, produto_row in pedido_data.iterrows():
        nome_produto = produto_row['Nome do Produto']
        preco = produto_row['Preço']
        quantidade = produto_row['Quantidade Pedida']
        produtos += f"- {nome_produto} (Quantidade: {quantidade}, Preço: {preco})\n"

    # Construir a mensagem
    message = (f"Olá! Recebemos seu pedido de bebidas, nº *{numero_pedido}* no BEES-Ambev 🐝.\n"
               f"Produtos no pedido:\n"
               f"{produtos}\n"
               f"Endereço de entrega:\n"
               f"{endereco},\n{cidade_uf},\n*CEP*: {cep}.\n"
               f"Forma de pagamento: {forma_pagamento}.*Não aceitamos forma de pagamento diferente da nota.*\n"
               f"Entrega de Segunda a Sábado, entre 8h e 18h.\n"
               f"Caso o endereço esteja incorreto, por favor abra uma solicitação de correção no app BEES.\n"
               f"Para confirmar digite *1*\n"
               f"Para cancelar digite *2*\n")

    # Enviar a mensagem para todos os números encontrados
    for phone in telefones:
        texto = urllib.parse.quote(message)
        link = f"https://web.whatsapp.com/send?phone={phone}&text={texto}"
        driver.get(link)
        
        # Aguarda até a interface do WhatsApp Web carregar completamente
        aguardar_carregamento(driver)

        try:
            time.sleep(5)  # Espera mais um pouco após o carregamento
            # Aguarda até que o botão de enviar esteja visível e habilitado
            send_button = WebDriverWait(driver, 10).until(
                EC.element_to_be_clickable((By.XPATH, '//button[@aria-label="Enviar"]'))
            )
            send_button.click()  # Envia a mensagem
            time.sleep(2)
            print(f"Mensagem enviada para {phone} - Pedido {numero_pedido}")

            # Marcar o pedido como 'Enviado' para evitar reenvio
            tabela_pedidos.loc[index, 'Status'] = 'Enviado'
            telefones_processados.add(phone)  # Adiciona telefone ao conjunto processado
        except Exception as e:
            print(f"Erro ao enviar mensagem para {phone} - Pedido {numero_pedido}")

# Salvar a tabela de pedidos atualizada com o status 'Enviado'
tabela_pedidos.to_csv(f'G:/Drives compartilhados/Cadastro BEES/MensagemEnviadas_{filial}_Atualizado.csv', index=False, encoding='utf-8-sig')
driver.quit()

Aguardando conexão com o WhatsApp Web...
Conectado ao WhatsApp Web.
Conversa carregada com sucesso.
Mensagem enviada para 11964617713 - Pedido 9196257507
Algum número do pedido 9196257507 já foi processado. Ignorando envio.
Conversa carregada com sucesso.
Mensagem enviada para 11910262908 - Pedido 9196249345
Algum número do pedido 9196249345 já foi processado. Ignorando envio.
Erro ao enviar mensagem para 13 - Pedido 9196248690
Erro ao enviar mensagem para 13 - Pedido 9196248690
Erro ao enviar mensagem para 13 - Pedido 9196248690
Erro ao enviar mensagem para 13 - Pedido 9196248690
Conversa carregada com sucesso.
Mensagem enviada para 21968217934 - Pedido 9196233787
Algum número do pedido 9196233787 já foi processado. Ignorando envio.
Erro ao enviar mensagem para 11400000000 - Pedido 9196232678
Erro ao enviar mensagem para 119377971440 - Pedido 9196232678
Erro ao enviar mensagem para 11043434343 - Pedido 9196232678
Erro ao enviar mensagem para 43900000000 - Pedido 9196216083
Conversa ca

KeyboardInterrupt: 

In [6]:
import pandas as pd
from openpyxl import load_workbook

# Definir as cidades do interior do RJ
cidades_interior = ['ITATIAIA', 'RESENDE', 'PORTO REAL', 'QUATIS', 'BARRA MANSA', 'VOLTA REDONDA', 'ARROZAL', 'PINHEIRAL', 'PIRAI', 'BARRA DO PIRAI', 'MENDES', 'VASSOURA', 'RIO CLARO']

# Remover duplicados
pedidos_unicos = set()

# Carregar o arquivo CSV
arquivo = pd.read_csv(f'G:/Drives compartilhados/Cadastro BEES/Pedidos_A_Preparar_{filial}.csv')

# Lista para armazenar os dados a serem salvos
dados_para_salvar = []

# Verificar pedidos com CPF e que sejam do estado do RJ e de cidades do interior
for index, row in arquivo.iterrows():
    if 'CPF' in row['Documento'] and 'RJ' in row['Cidade/UF'] and any(cidade in row['Cidade/UF'] for cidade in cidades_interior):
        pedidos_unicos.add(row['Numero Pedido'])

# Coletar dados para salvar no arquivo
for pedido in pedidos_unicos:
    # Filtra a linha do pedido específico
    pedido_info = arquivo[arquivo['Numero Pedido'] == pedido].iloc[0]
    
    # Adiciona as informações desejadas na lista
    dados_para_salvar.append({
        'Numero Pedido': pedido_info['Numero Pedido'],
        'Documento': pedido_info['Documento'],
        'Cidade/UF': pedido_info['Cidade/UF'],
        'Data Pedido': pedido_info['Data Pedido'],
        'Data Entrega': pedido_info['Data Entrega'],
        'Nome Comercial': pedido_info['Nome Comercial'],
        'Endereço de Entrega': pedido_info['Endereço de Entrega'],
        'CEP': pedido_info['CEP'],
        'Forma de Pagamento': pedido_info['Forma de Pagamento'],
        'Contato 1': pedido_info['Contato 1'],
        'Contato 2': pedido_info['Contato 2'],
        'Contato 3': pedido_info['Contato 3'],
          
    })

# Criar um DataFrame a partir dos dados coletados
df = pd.DataFrame(dados_para_salvar)

# Nome do arquivo Excel
file_name = f'Pedidos_Selecionados_{filial}.xlsx'

# Se o arquivo já existir, abrir e adicionar os dados, caso contrário, criar um novo arquivo
try:
    # Tenta abrir o arquivo existente
    with pd.ExcelWriter(file_name, mode='a', engine='openpyxl', if_sheet_exists='overlay') as writer:
        df.to_excel(writer, index=False, sheet_name='Pedidos', header=False)  # Não inclui cabeçalho se já existir
    print("Dados acrescentados ao arquivo com sucesso!")
except FileNotFoundError:
    # Se o arquivo não existir, cria um novo
    df.to_excel(file_name, index=False, sheet_name='Pedidos')
    print("Arquivo Excel criado com sucesso!")


Arquivo Excel criado com sucesso!
